# Hospital Cluster Analysis for Top Indian Cities for COVID-19 Control Room Setup

In [9]:
# Importing Packages
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
import folium

##### Hiding sensetive code

In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
# Retrieving Hospital details using Foresquare API
LIMIT = 100
cities = ["Delhi, Delhi", 'Mumbai, Maharashtra', 'Kolkata, West Bengal', 'Chennai, Tamilnadu', 'Bangalore, Karnataka', 'Hyderabad, Telangana']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d196941735")
    results[city] = requests.get(url).json()

In [12]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [17]:
# Ploting Hospital locations in each city
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # adding markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of hospitals in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of hospitals in Delhi, Delhi =  37
Showing Top 100
Total number of hospitals in Mumbai, Maharashtra =  71
Showing Top 100
Total number of hospitals in Kolkata, West Bengal =  31
Showing Top 100
Total number of hospitals in Chennai, Tamilnadu =  52
Showing Top 100
Total number of hospitals in Bangalore, Karnataka =  68
Showing Top 100
Total number of hospitals in Hyderabad, Telangana =  40
Showing Top 100


### Displaying Hospital location in each city

In [18]:
print(f"Total number of hospitals in {cities[0]} = ", results[cities[0]]['response']['totalResults'])
maps[cities[0]]

Total number of hospitals in Delhi, Delhi =  37


In [19]:
print(f"Total number of hospitals in {cities[1]} = ", results[cities[1]]['response']['totalResults'])
maps[cities[1]]

Total number of hospitals in Mumbai, Maharashtra =  71


In [21]:
print(f"Total number of hospitals in {cities[2]} = ", results[cities[2]]['response']['totalResults'])
maps[cities[2]]

Total number of hospitals in Kolkata, West Bengal =  31


In [22]:
print(f"Total number of hospitals in {cities[3]} = ", results[cities[3]]['response']['totalResults'])
maps[cities[3]]

Total number of hospitals in Chennai, Tamilnadu =  52


In [23]:
print(f"Total number of hospitals in {cities[4]} = ", results[cities[4]]['response']['totalResults'])
maps[cities[4]]

Total number of hospitals in Bangalore, Karnataka =  68


In [25]:
print(f"Total number of hospitals in {cities[5]} = ", results[cities[5]]['response']['totalResults'])
maps[cities[5]]

Total number of hospitals in Hyderabad, Telangana =  40


### Calculating mean distance between Hospitals in each cities

In [26]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Delhi, Delhi
Mean Distance from Mean coordinates
0.07314153493753241
Mumbai, Maharashtra
Mean Distance from Mean coordinates
0.08008461412161624
Kolkata, West Bengal
Mean Distance from Mean coordinates
0.051839274624414594
Chennai, Tamilnadu
Mean Distance from Mean coordinates
0.046351385806275396
Bangalore, Karnataka
Mean Distance from Mean coordinates
0.05984544445196689
Hyderabad, Telangana
Mean Distance from Mean coordinates
0.049969500037487284


### Displaying the mean distance between Hospitals as an optimum spot for setting up COVID-19 Control Room

In [27]:
print(cities[0])
maps[cities[0]]

Delhi, Delhi


In [28]:
print(cities[1])
maps[cities[1]]

Mumbai, Maharashtra


In [29]:
print(cities[2])
maps[cities[2]]

Kolkata, West Bengal


In [30]:
print(cities[3])
maps[cities[3]]

Chennai, Tamilnadu


In [31]:
print(cities[4])
maps[cities[4]]

Bangalore, Karnataka


In [32]:
print(cities[5])
maps[cities[5]]

Hyderabad, Telangana
